In [ ]:
from flask import Flask, render_template, request
from wtforms import (Form, FileField, StringField, TextAreaField, validators, SubmitField, IntegerField, MultipleFileField)
from keras.models import load_model
import tensorflow as tf
import html
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pdfplumber
from docx2pdf import convert
import os
import pandas as pd
import pythoncom

# Create app
app = Flask(__name__)

class ReusableForm1(Form):
    """Document Classifier - DCNN Model"""
    #Document name
    docName = StringField(u'Enter document name:', validators=[
                     validators.InputRequired()])
    # Document content
    docContent = TextAreaField(u'Enter document content:', validators=[
                     validators.InputRequired()])
    
    # Submit button
    submit1 = SubmitField("Submit")

class ReusableForm2(Form):
    """Document Classifier - DCNN Model"""
    #Document name
    docFiles = MultipleFileField(u'', validators=[
                     validators.InputRequired()])
    
    # Submit button
    submit2 = SubmitField("Submit")
    
def load_dcnn_model():
    """Load in the pre-trained model"""
    global cnnmodel
    cnnmodel = load_model('DCNN_Model/dcnnmodel')
    

def cleanTextAndEmbedings(docContent):
    df = pd.DataFrame({'content': [docContent]})
    #1. Remove newline characters
    df = df.replace(r'\n', '', regex=True)
    #2.Remove spcial characters
    spec_chars = ["!", '"', "#", "%", "&", "'", "(", ")", "*", "+", ",", "-", ".", "/", ":",";", "<", "=", ">", "?", "@", "[", "\\", "]", "^", "_", "`", "~", "{", "|", "}"]
    for char in spec_chars:
        df['content'] = df['content'].str.replace(char, ' ')
    #3.HTML escape characters
    df = html.escape(df)
    #5. Convert all letters to lowercase
    df['content'] = df['content'].str.lower()
    #6. Remove stopwords
    stop_words = set(stopwords.words('english'))
    lst_contents = []
    for contents in df['content']:
        word_tokens = word_tokenize(contents)
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        txt_sentence = " ".join(filtered_sentence)
        lst_contents.append(txt_sentence)
    df['content_without_stopwords'] = lst_contents

    tokenizer = Tokenizer(num_words=5000,lower=False)
    tokenizer.fit_on_texts(df['content_without_stopwords'])
    X_test = tokenizer.texts_to_sequences(df['content_without_stopwords'])
    X_test = pad_sequences(X_test, padding='post', maxlen=10)
    return X_test

def extractTextFromPDF(fileName):
    textContent=''
    with pdfplumber.open(fileName) as pdf:
        for page in pdf.pages:
            textContent += page.extract_text()
    return textContent

def covertDOCXToPDF(filename):
    split_tup = os.path.splitext(filename)
    file_name = split_tup[0]
    file_extension = split_tup[1]
    pythoncom.CoInitialize()
    convert(filename)
    return file_name+".pdf"

def extractTextFromExcel(filename):
    df = pd.read_excel(filename, index_col=0)  
    textContent=df.to_string()
    return textContent

# Home page
@app.route("/index", methods=['GET', 'POST'])
def index():
    """Home page of app with form"""
    # Create form
    form1 = ReusableForm1(request.form)
    docCategory=''
    # On form entry and all conditions met
    if request.method == 'POST' and form1.validate():
        # Extract information
        docName = request.form['docName']
        docContent = request.form['docContent']        
        if docName is not None and docContent is not None:
            X_test = cleanTextAndEmbedings(docContent);
            y_pred=cnnmodel.predict(X_test)
            print("y_pred: ", y_pred)
            if y_pred.argmax()==0:
                docCategory='Public'
            if y_pred.argmax()==1:
                docCategory='Private'
            if y_pred.argmax()==2:
                docCategory='Confidential'
            print("docCategory: "+docCategory)
            return render_template('classification_result.html', input=docCategory)           

    # Send template information to index.html
    return render_template('index.html', form=form1)

@app.route('/selectfiles', methods=['GET', 'POST'])
def selectfiles():
    # Create form
    form2 = ReusableForm2(request.form)
    repName='./Docs/'
    docCategory=''
    data = []
    # On form entry and all conditions met
    if request.method == 'POST' and form2.validate():
        if form2.docFiles.data:
            files = form2.docFiles.data
            for file in files:
                split_tup = os.path.splitext(file)
                file_name = split_tup[0]
                file_extension = split_tup[1]
                print(file_extension)
                docContent=''
                if file_extension== '.pdf':
                    docContent = extractTextFromPDF(repName+file)
                elif file_extension== '.docx':
                    convertedFile = covertDOCXToPDF(repName+file)
                    docContent = extractTextFromPDF(convertedFile)
                    os.remove(convertedFile)
                elif file_extension== '.xlsx':
                    docContent = extractTextFromExcel(repName+file)
                        
                X_test = cleanTextAndEmbedings(docContent)
                y_pred=cnnmodel.predict(X_test)
                print("y_pred: ", y_pred)
                if y_pred.argmax()==0:
                    docCategory='Public'
                if y_pred.argmax()==1:
                    docCategory='Private'
                if y_pred.argmax()==2:
                    docCategory='Confidential'
                print("docCategory: "+docCategory)
                data.append([file, docCategory])
            df = pd.DataFrame(data, columns=['File name', 'Category'])
            print(df)
            return render_template('multiclassification_result.html',  tables=[df.to_html(classes='data')], titles=df.columns.values)
               
    return render_template('selectfiles.html', form=form2)

if __name__ == "__main__":
    load_dcnn_model()
    app.run(debug=True)

Loading DCNN and Flask starting server...please wait until server has fully started
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.197:9090/ (Press CTRL+C to quit)
192.168.0.197 - - [13/Apr/2022 07:52:03] "GET / HTTP/1.1" 404 -
192.168.0.197 - - [13/Apr/2022 07:52:04] "GET /favicon.ico HTTP/1.1" 404 -
192.168.0.197 - - [13/Apr/2022 07:52:10] "GET /selectfiles HTTP/1.1" 200 -
192.168.0.197 - - [13/Apr/2022 07:52:10] "GET /static/css/main.css HTTP/1.1" 200 -
192.168.0.197 - - [13/Apr/2022 07:52:10] "GET /static/images/script_background.png HTTP/1.1" 404 -


.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\disha\AppData\Local\Temp/ipykernel_15624/185939347.py:56: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['content'] = df['content'].str.replace(char, ' ')


...

y_pred:  [[0.7794057  0.37714878 0.3352645 ]]
docCategory: Public
.pdf
y_pred:  [[0.5504359  0.56321067 0.55313474]]
docCategory: Private
.docx


  0%|          | 0/1 [00:00<?, ?it/s]

192.168.0.197 - - [13/Apr/2022 08:15:23] "POST /selectfiles HTTP/1.1" 200 -
192.168.0.197 - - [13/Apr/2022 08:15:23] "GET /static/images/script_background.png HTTP/1.1" 404 -


y_pred:  [[0.2513851  0.26724997 0.623335  ]]
docCategory: Confidential
                           File name      Category
0  Deep Learning Theory Syllabus.pdf        Public
1             Prerequisite Marks.pdf       Private
2                 Salary-Slip-2.docx  Confidential


192.168.0.197 - - [13/Apr/2022 08:31:51] "GET /selectfiles HTTP/1.1" 200 -
192.168.0.197 - - [13/Apr/2022 08:31:51] "GET /static/images/script_background.png HTTP/1.1" 404 -
